## Rubric

Instructions: DELETE this cell before you submit via a `git push` to your repo before deadline. This cell is for your reference only and is not needed in your report. 

Scoring: Out of 10 points

- Each Developing  => -2 pts
- Each Unsatisfactory/Missing => -4 pts
  - until the score is 

If students address the detailed feedback in a future checkpoint they will earn these points back


|                  | Unsatisfactory                                                                                                                                                                                                    | Developing                                                                                                                                                                                              | Proficient                                     | Excellent                                                                                                                              |
|------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------|
| Data relevance   | Did not have data relevant to their question. Or the datasets don't work together because there is no way to line them up against each other. If there are multiple datasets, most of them have this trouble | Data was only tangentially relevant to the question or a bad proxy for the question. If there are multiple datasets, some of them may be irrelevant or can't be easily combined.                       | All data sources are relevant to the question. | Multiple data sources for each aspect of the project. It's clear how the data supports the needs of the project.                         |
| Data description | Dataset or its cleaning procedures are not described. If there are multiple datasets, most have this trouble                                                                                              | Data was not fully described. If there are multiple datasets, some of them are not fully described                                                                                                      | Data was fully described                       | The details of the data descriptions and perhaps some very basic EDA also make it clear how the data supports the needs of the project. |
| Data wrangling   | Did not obtain data. They did not clean/tidy the data they obtained.  If there are multiple datasets, most have this trouble                                                                                 | Data was partially cleaned or tidied. Perhaps you struggled to verify that the data was clean because they did not present it well. If there are multiple datasets, some have this trouble | The data is cleaned and tidied.                | The data is spotless and they used tools to visualize the data cleanliness and you were convinced at first glance                      |


# COGS 108 - Data Checkpoint

## Authors

Omar Abbasi: Project Administration, Conceptualization, Formal Analysis, Visualization, Writing – Original Draft  
Zahir Ali: Conceptualization, Visualization, Software, Writing – Reviewing/Edits  
Adam Hamadene: Research, Formal Analysis  
Mostafa Darwish: Visualization, Writing – Original Draft, Data Curation  
Yasir Rizvi: Data Cleaning 


## Research Question

How do funding size, industry sector, and geographic location influence both the likelihood and timing of startup failure versus acquisition?



## Background and Prior Work

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback

## Hypothesis


Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback


## Data

### Data overview

- **Dataset #1**: See section below for details, loading, and wrangling steps.

- **Dataset #2**
  - **Dataset Name**: Big Startup Success/Fail Dataset from Crunchbase
  - **Link to the dataset**: `https://www.kaggle.com/datasets/yanmaksi/big-startup-secsees-fail-dataset-from-crunchbase`
  - **Number of observations**: Computed in the code cell below (after loading)
  - **Number of variables**: Computed in the code cell below (after loading)
  - **Most relevant variables for our project**:
    - Funding-related: `funding_total_usd` (or similar), `funding_rounds`, `last_funding_at`
    - Outcome/status: `status` (e.g., acquired/ipo/operating/closed), `acquired_at`, `closed_at`
    - Company profile: `name`, `category`/`industry`, `country`, `region`/`city`, `num_employees`
    - Timing: `founded_at` (and event dates as above) for computing time-to-failure or time-to-acquisition
  - **Shortcomings / caveats**:
    - Potential label noise: outcomes may be simplified; “operating” does not guarantee long-term success; “closed” labels may lag reality
    - Missing or inconsistent dates across companies, which affects survival/time-to-event analyses
    - Possible duplicates or multiple rows for the same company over time; category/industry taxonomy can be messy
    - Survivorship and reporting bias inherent to Crunchbase; geographic and sector coverage is uneven

If we combine datasets, we will align on common keys (e.g., standardized company names and/or website domains), and normalize geography (country/region) and industry taxonomies before joining.

In [ ]:
# Run this code every time when you're actively developing modules in .py files.  It's not needed if you aren't making modules
#
## this code is necessary for making sure that any modules we load are updated here 
## when their source code .py files are modified

%load_ext autoreload
%autoreload 2

In [ ]:
# Setup code -- this only needs to be run once after cloning the repo!
# this code downloads the data from its source to the `data/00-raw/` directory
# if the data hasn't updated you don't need to do this again!

# if you don't already have these packages (you should!) uncomment this line
# %pip install requests tqdm

import sys
sys.path.append('./modules') # this tells python where to look for modules to import

import get_data # this is where we get the function we need to download data

# replace the urls and filenames in this list with your actual datafiles
# yes you can use Google drive share links or whatever
# format is a list of dictionaries; 
# each dict has keys of 
#   'url' where the resource is located
#   'filename' for the local filename where it will be stored 
datafiles = [
    { 'url': 'https://raw.githubusercontent.com/fivethirtyeight/data/refs/heads/master/airline-safety/airline-safety.csv', 'filename':'airline-safety.csv'},
    { 'url': 'https://raw.githubusercontent.com/fivethirtyeight/data/refs/heads/master/bad-drivers/bad-drivers.csv', 'filename':'bad-drivers.csv'}
]

get_data.get_raw(datafiles,destination_directory='data/00-raw/')

### Dataset #1 

Instructions: 
1. Change the header from Dataset #1 to something more descriptive of the dataset
2. Write a few paragraphs about this dataset. Make sure to cover
   1. Describe the important metrics, what units they are in, and giv some sense of what they mean.  For example "Fasting blood glucose in units of mg glucose per deciliter of blood.  Normal values for healthy individuals range from 70 to 100 mg/dL.  Values 100-125 are prediabetic and values >125mg/dL indicate diabetes. Values <70 indicate hypoglycemia. Fasting idicates the patient hasn't eaten in the last 8 hours.  If blood glucose is >250 or <50 at any time (regardless of the time of last meal) the patient's life may be in immediate danger"
   2. If there are any major concerns with the dataset, describe them. For example "Dataset is composed of people who are serious enough about eating healthy that they voluntarily downloaded an app dedicated to tracking their eating patterns. This sample is likely biased because of that self-selection. These people own smartphones and may be healthier and may have more disposable income than the average person.  Those who voluntarily log conscientiously and for long amounts of time are also likely even more interested in health than those who download the app and only log a bit before getting tired of it"
3. Use the cell below to 
    1. load the dataset 
    2. make the dataset tidy or demonstrate that it was already tidy
    3. demonstrate the size of the dataset
    4. find out how much data is missing, where its missing, and if its missing at random or seems to have any systematic relationships in its missingness
    5. find and flag any outliers or suspicious entries
    6. clean the data or demonstrate that it was already clean.  You may choose how to deal with missingness (dropna of fillna... how='any' or 'all') and you should justify your choice in some way
    7. You will load raw data from `data/00-raw/`, you will (optionally) write intermediate stages of your work to `data/01-interim` and you will write the final fully wrangled version of your data to `data/02-processed`
4. Optionally you can also show some summary statistics for variables that you think are important to the project
5. Feel free to add more cells here if that's helpful for you


In [ ]:
## YOUR CODE TO LOAD/CLEAN/TIDY/WRANGLE THE DATA GOES HERE


### Startup Success/Fail Dataset from Crunchbase

This dataset aggregates company-level records from Crunchbase, including basic profile fields (name, industry/category, geography), funding history (e.g., number of rounds, last funding date, total funding), and outcome indicators (e.g., operating, acquired, IPO, closed). These variables are directly relevant to our research question because they allow us to:

- Quantify funding size and intensity via `funding_total_usd` and `funding_rounds`.
- Identify industry sector using category taxonomy fields.
- Place companies geographically using country and region/city fields.
- Measure both the likelihood and timing of outcomes using `status` along with event dates such as `acquired_at` and `closed_at`.

Key metrics and units, with interpretation:
- **Funding totals (USD)**: Continuous financial magnitude; heavy-tailed and requires robust treatment (log transform or winsorization).
- **Funding rounds (count)**: Discrete intensity of investor validation; potentially confounded by time and sector norms.
- **Status / outcome (categorical)**: Typical labels include operating, acquired, IPO, closed. For analysis, we will map to success (acquired or IPO) and failure (closed). Operating can be treated as right-censored in time-to-event analyses.
- **Dates (ISO-like strings)**: `founded_at`, `last_funding_at`, `acquired_at`, `closed_at`. We will parse to datetime and compute durations, e.g., time from founding to acquisition/closure.

Major concerns and data limitations:
- **Labeling and censoring**: “Operating” is not the same as success; many operating companies will eventually fail or be acquired. We will treat these as censored when modeling time-to-event.
- **Missingness / inconsistency**: Dates, industries, and geographies are often missing or non-standard. We will profile missingness and document any imputation or row-dropping.
- **Duplicates / entity resolution**: Company names can collide; we will check for likely duplicates and standardize names to reduce double counting.
- **Bias**: Crunchbase coverage is uneven by geography and sector, and reporting incentives differ by outcome and company size.

We will follow the same wrangling standards as Dataset #1: load from `data/00-raw/`, tidy column names, validate types, inspect missingness systematically, flag outliers in funding, derive outcome labels and time-to-event targets, and save a clean, analysis-ready CSV to `data/02-processed/`.

In [ ]:
# Crunchbase Startup Success/Fail — Load, Clean, Tidy, Wrangle
#
# Notes:
# - Place the Kaggle CSV from the dataset into `data/00-raw/`.
# - If multiple CSVs exist, this cell attempts to auto-detect the most likely file.
# - Adjust `RAW_FILENAME_OVERRIDE` if auto-detection fails.

from __future__ import annotations

from pathlib import Path
from typing import Optional, List, Dict

import pandas as pd
import numpy as np

RAW_DIR = Path("data/00-raw")
INTERIM_DIR = Path("data/01-interim")
PROCESSED_DIR = Path("data/02-processed")

INTERIM_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# If you know the exact filename, set it here; otherwise keep as None
RAW_FILENAME_OVERRIDE: Optional[str] = "dataset2.csv"  # renamed for consistency

# Heuristics for picking likely files
LIKELY_SUBSTRINGS: List[str] = [
    "crunchbase",
    "startup",
    "startups",
    "success",
    "fail",
]


def pick_raw_file(raw_dir: Path, override: Optional[str]) -> Path:
    """Pick the most likely raw CSV file for the Crunchbase dataset.

    Args:
        raw_dir: Directory containing raw files.
        override: Explicit filename if known.

    Returns:
        Path to the selected CSV file.

    Raises:
        FileNotFoundError: If no suitable file is found.
    """
    if override is not None:
        candidate = raw_dir / override
        if candidate.exists():
            return candidate
        raise FileNotFoundError(
            f"Override file not found: {candidate}. Place it in {raw_dir} or update RAW_FILENAME_OVERRIDE."
        )

    csvs = sorted(raw_dir.glob("*.csv"))
    if not csvs:
        raise FileNotFoundError(
            f"No CSV files found in {raw_dir}. Download the Kaggle dataset and place the CSV here."
        )

    # Rank by substring match then by size (descending)
    def score(p: Path) -> tuple[int, int]:
        name = p.name.lower()
        match_score = sum(1 for s in LIKELY_SUBSTRINGS if s in name)
        size_score = p.stat().st_size
        return (match_score, size_score)

    ranked = sorted(csvs, key=score, reverse=True)
    return ranked[0]


raw_path = pick_raw_file(RAW_DIR, RAW_FILENAME_OVERRIDE)
print(f"Selected raw file: {raw_path}")

# Load data
# Use low_memory=False for mixed types, and keep original column names for inspection before standardization
raw_df = pd.read_csv(raw_path, low_memory=False)
print(f"Raw shape: {raw_df.shape}")

# Standardize column names: snake_case
raw_df.columns = (
    raw_df.columns.str.strip().str.replace(" ", "_", regex=False).str.replace("-", "_", regex=False).str.lower()
)

# Basic de-duplication by exact row match and by name if present
if "name" in raw_df.columns:
    before = len(raw_df)
    raw_df = raw_df.drop_duplicates()
    raw_df = raw_df.drop_duplicates(subset=["name"], keep="first")
    print(f"Dropped duplicates -> {before} -> {len(raw_df)} rows")
else:
    raw_df = raw_df.drop_duplicates()

# Identify key columns by fuzzy names
def find_col(candidates: List[str]) -> Optional[str]:
    cols = set(raw_df.columns)
    for c in candidates:
        if c in cols:
            return c
    return None

col_name = find_col(["name", "company", "company_name"])
col_status = find_col(["status", "state", "current_status"])  # typical outcome label
col_funded_total = find_col(["funding_total_usd", "funding_total", "total_funding_usd", "total_funding"])
col_rounds = find_col(["funding_rounds", "num_funding_rounds", "rounds"])
col_category = find_col(["category", "category_list", "industry", "category_groups"])
col_country = find_col(["country", "country_code", "country_iso", "country_name"])
col_region = find_col(["region", "state_region", "state", "city", "location"])
col_founded = find_col(["founded_at", "founded", "founded_date"])  # date-like
col_acquired = find_col(["acquired_at", "acquired", "acquisition_date"])  # date-like
col_closed = find_col(["closed_at", "closed", "closing_date"])  # date-like
col_last_funding = find_col(["last_funding_at", "last_funding_date"])  # date-like
col_employees = find_col(["num_employees", "employee_count", "employees"])

# Restrict to relevant columns (retain existing only)
keep_cols: List[str] = [
    c
    for c in [
        col_name,
        col_status,
        col_funded_total,
        col_rounds,
        col_category,
        col_country,
        col_region,
        col_founded,
        col_acquired,
        col_closed,
        col_last_funding,
        col_employees,
    ]
    if c is not None
]

work_df = raw_df[keep_cols].copy()
print(f"Working shape (selected columns): {work_df.shape}")

# Parse dates where present
for dcol in [col_founded, col_acquired, col_closed, col_last_funding]:
    if dcol is not None and dcol in work_df.columns:
        work_df[dcol] = pd.to_datetime(work_df[dcol], errors="coerce")

# Coerce funding to numeric USD if present
if col_funded_total is not None and col_funded_total in work_df.columns:
    # Remove currency symbols/commas if any
    work_df[col_funded_total] = (
        work_df[col_funded_total]
        .astype(str)
        .str.replace(",", "", regex=False)
        .str.replace("$", "", regex=False)
        .str.strip()
    )
    work_df[col_funded_total] = pd.to_numeric(work_df[col_funded_total], errors="coerce")

# Derive outcome label: success/failure/censored
# success: acquired or IPO; failure: closed; censored: operating/others with no terminal date
def derive_outcome(status_val: Optional[str]) -> Optional[str]:
    if status_val is None or pd.isna(status_val):
        return None
    s = str(status_val).strip().lower()
    if any(tok in s for tok in ["acquired", "ipo"]):
        return "success"
    if "closed" in s:
        return "failure"
    if any(tok in s for tok in ["operating", "active"]):
        return "censored"
    return None

if col_status is not None and col_status in work_df.columns:
    work_df["outcome_label"] = work_df[col_status].map(derive_outcome)
else:
    work_df["outcome_label"] = None

# Time-to-event from founded to first terminal event if present
def pick_event_date(row: pd.Series) -> pd.Timestamp | pd.NaT:
    # prefer acquired/closed dates; else NaT
    acquired_date = row[col_acquired] if col_acquired in row and pd.notna(row[col_acquired]) else pd.NaT
    closed_date = row[col_closed] if col_closed in row and pd.notna(row[col_closed]) else pd.NaT
    # pick whichever is earliest non-NaT
    if pd.notna(acquired_date) and pd.notna(closed_date):
        return min(acquired_date, closed_date)
    if pd.notna(acquired_date):
        return acquired_date
    if pd.notna(closed_date):
        return closed_date
    return pd.NaT

if col_founded is not None and col_founded in work_df.columns:
    work_df["event_date"] = work_df.apply(pick_event_date, axis=1)
    work_df["time_to_event_days"] = (
        work_df["event_date"] - work_df[col_founded]
    ).dt.days
else:
    work_df["event_date"] = pd.NaT
    work_df["time_to_event_days"] = np.nan

# Missingness profile
missing_pct = work_df.isna().mean().sort_values(ascending=False)
print("\nMissingness (% of rows missing per column):")
print((missing_pct * 100).round(1))

# Flag outliers in funding (IQR method)
if col_funded_total is not None and col_funded_total in work_df.columns:
    fund = work_df[col_funded_total].dropna()
    if len(fund) > 0:
        q1, q3 = np.percentile(fund, [25, 75])
        iqr = q3 - q1
        upper = q3 + 1.5 * iqr
        lower = max(0, q1 - 1.5 * iqr)
        work_df["funding_outlier_flag"] = (
            (work_df[col_funded_total] < lower) | (work_df[col_funded_total] > upper)
        )
        print(
            f"Funding IQR bounds -> lower={lower:,.0f}, upper={upper:,.0f}; "
            f"outliers flagged: {work_df['funding_outlier_flag'].sum()}"
        )
    else:
        work_df["funding_outlier_flag"] = False
else:
    work_df["funding_outlier_flag"] = False

# Minimal cleaning: drop rows with no name and no status
min_keep_cols = [c for c in [col_name, col_status] if c is not None]
if min_keep_cols:
    before = len(work_df)
    work_df = work_df.dropna(subset=min_keep_cols, how="all")
    print(f"Dropped rows missing all of {min_keep_cols}: {before} -> {len(work_df)}")

# Save processed
processed_path = PROCESSED_DIR / "startups_crunchbase_processed.csv"
work_df.to_csv(processed_path, index=False)
print(f"\nProcessed file written to: {processed_path}")

# Summary
print("\nSummary:")
print({
    "n_rows": len(work_df),
    "n_cols": work_df.shape[1],
    "outcome_counts": work_df["outcome_label"].value_counts(dropna=False).to_dict() if "outcome_label" in work_df else {},
})


Selected raw file: data/00-raw/big_startup_secsees_dataset.csv
Raw shape: (66368, 14)
Dropped duplicates -> 66368 -> 66103 rows
Working shape (selected columns): (66103, 9)

Missingness (% of rows missing per column):
event_date            100.0
time_to_event_days    100.0
founded_at             22.9
funding_total_usd      19.2
region                 12.1
country_code           10.5
category_list           4.7
name                    0.0
status                  0.0
funding_rounds          0.0
last_funding_at         0.0
outcome_label           0.0
dtype: float64
Funding IQR bounds -> lower=0, upper=24,495,250; outliers flagged: 7363
Dropped rows missing all of ['name', 'status']: 66103 -> 66103

Processed file written to: data/02-processed/startups_crunchbase_processed.csv

Summary:
{'n_rows': 66103, 'n_cols': 13, 'outcome_counts': {'censored': 52812, 'success': 7088, 'failure': 6203}}


## Ethics

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback

## Team Expectations 

In regards to communication, we plan on using iMessage to text one anotehr. We believe this is one of the most efficient and easiest ways for us to contact one another. As for the ,essaging itself, all members have agreed to expect a rresponse, whether that be a message or a reaction, from each group member within 3 hours of the initial text being sent. We will meet twice per week, every Monday morning we will reserve a study space in the Geisel Library to meet and prodvide updates, followed by a Zoom meetimng every Friday afternoon to consolidate the designated progress from Onday's meeting. In regarcs to tone, we agree to all expect respectful interactions. Even when disagreement takes place, the person expressing the lack of approval should explain their reasonibg, as well as an alkternative method that they beleive is better. We will be concise and to the point, but still maintaining respect for one another and everyone's ideas. We plan to use voting to make decisions as a group, especially for disagreements and changes to our original plan. The project administrator will be in charge of calling teh vote, ad we will go with the majority ruling, as wel have3 5 people. We will not accept abstaining from votes. We do have specialized roles for each person, hwoever, because there is overlap, we do plan to share a lot of the responsibilties. As we are a team, we plan on heloing each other out when possible, especially if one person is struggling with a specific task. We assign roles and tasks based on the skillsets of the members, which we have laready discussed in detail. We have set a policy that struggles are inevitbale, as we are all busy. We have a guideline that whenever someone is falling behind, there is no hassle or problem in expressing that as early as possible. We would rather know what to fix earlier on in the process, rather than have a last minute lack of execution. Struggles with certain tasks hoild be expressd immediately, as we will set egos aside ti help regardless of role/assignned tasks, in roder to priotitize the team as a unit/whole.


## Project Timeline Proposal

Instructions: Replace this with your timeline.  **PLEASE UPDATE your Timeline!** No battle plan survives contact with the enemy, so make sure we understand how your plans have changed.  Also if you have lost points on the previous checkpoint fix them